<a href="https://www.kaggle.com/code/chongyockeng/ie4483-catdog?scriptVersionId=205201206" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split

import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.transforms import InterpolationMode
from torchinfo import summary

from timeit import default_timer as timer 
from tqdm.auto import tqdm

import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/kaggle/input/custom-packages/')
import data_setup, engine, model, utils

from utils import CLAHETransform, create_writer, save_model
from data_setup import create_dataloaders
from model import create_vgg11, create_vgg16, create_vgg19

In [3]:
config = {"BATCH_SIZE":32,
         "LEARNING_RATE":1e-4}

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
base_dir = "/kaggle/input/final-project-dataset/datasets"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)

Train Directory: /kaggle/input/final-project-dataset/datasets/train
Validation Directory: /kaggle/input/final-project-dataset/datasets/val
Test Directory: /kaggle/input/final-project-dataset/datasets/test


In [6]:

# Case 1: VGG Default Transform
transform_case_1 = torchvision.models.VGG19_Weights.DEFAULT.transforms()

# Case 2: CLAHE
transform_case_2 = transforms.Compose([
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),  # Apply CLAHE
    transforms.Resize((256, 256), # Resize to 256x256
    interpolation=InterpolationMode.BICUBIC),  
    transforms.CenterCrop(224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Case 3: Rotation + Horizontal Flip
transform_case_3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 

])
# Case 4: CLAHE + Rotation + Horizontal Flip
transform_case_4 = transforms.Compose([
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),  # Apply CLAHE
    transforms.Resize((256, 256), # Resize to 256x256
    interpolation=InterpolationMode.BICUBIC),  
    transforms.CenterCrop(224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
def create_dataloaders_helper(transformation):
    return create_dataloaders(train_dir=train_dir,val_dir=val_dir,transform=transformation,
                              batch_size=config['BATCH_SIZE'])

In [8]:
experiment_dataloaders = {
                        "Default Transform": create_dataloaders_helper(transform_case_1),
                        "CLAHE Transform": create_dataloaders_helper(transform_case_2),
                        "Rotation + Horizontal Flip Transform": create_dataloaders_helper(transform_case_3),
                        "CLAHE + Rotation + Horizontal Flip Transform": create_dataloaders_helper(transform_case_4),
                        }

In [9]:
# 1. Set the random seeds
torch.manual_seed(42)

models = ["vgg11","vgg16","vgg19"]

num_epochs = [5,10]
# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each DataLoader
for dataloader_name, dataloaders in experiment_dataloaders.items():
    class_names = dataloaders[-1]
    # 4. Loop through each number of epochs
    for epochs in num_epochs: 

        # 5. Loop through each model name and create a new model based on the name
        for model_name in models:

            # 6. Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # 7. Select the model
            if model_name == "vgg11":
                model = create_vgg11(device=device,class_names=class_names) 
            elif model_name == "vgg16":
                model = create_vgg16(device=device,class_names=class_names)
            else:
                model = create_vgg19(device=device,class_names=class_names) 
            
            # 8. Create a new loss and optimizer for every model
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=config["LEARNING_RATE"])

            # 9. Train target model with target dataloaders and track experiments
            engine.train(model=model,
                  train_dataloader=dataloaders[0],
                  test_dataloader=dataloaders[1], 
                  val_dataloader=dataloaders[2],
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=epochs,
                  device=device,
                  writer=create_writer(experiment_name=dataloader_name,
                                       model_name=model_name,
                                       extra=f"{epochs}_epochs"))
            
            # 10. Save the model to file so we can get back the best model
            save_filepath = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: vgg11
[INFO] DataLoader: Default Transform
[INFO] Number of epochs: 5


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:02<00:00, 189MB/s]


[INFO] Created new VGG11 model.
[INFO] Created SummaryWriter, saving to: logs/2024-11-04/Default Transform/vgg11/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1533 | train_acc: 0.9596 | test_loss: 0.0673 | test_acc: 0.9785
Epoch: 2 | train_loss: 0.0441 | train_acc: 0.9851 | test_loss: 0.0652 | test_acc: 0.9800
Epoch: 3 | train_loss: 0.0167 | train_acc: 0.9949 | test_loss: 0.0856 | test_acc: 0.9798
Epoch: 4 | train_loss: 0.0166 | train_acc: 0.9945 | test_loss: 0.1141 | test_acc: 0.9795
Epoch: 5 | train_loss: 0.0328 | train_acc: 0.9920 | test_loss: 0.1450 | test_acc: 0.9778
[INFO] Saving model to: models/vgg11_Default Transform_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: vgg16
[INFO] DataLoader: Default Transform
[INFO] Number of epochs: 5


...

Epoch: 1 | train_loss: 0.1705 | train_acc: 0.9598 | test_loss: 0.0791 | test_acc: 0.9765
Epoch: 2 | train_loss: 0.0392 | train_acc: 0.9878 | test_loss: 0.1257 | test_acc: 0.9683
Epoch: 3 | train_loss: 0.0163 | train_acc: 0.9942 | test_loss: 0.1202 | test_acc: 0.9755
Epoch: 4 | train_loss: 0.0225 | train_acc: 0.9939 | test_loss: 0.1989 | test_acc: 0.9752
Epoch: 5 | train_loss: 0.0461 | train_acc: 0.9903 | test_loss: 0.1933 | test_acc: 0.9692
Epoch: 6 | train_loss: 0.0136 | train_acc: 0.9958 | test_loss: 0.2012 | test_acc: 0.9748
Epoch: 7 | train_loss: 0.0161 | train_acc: 0.9959 | test_loss: 0.1991 | test_acc: 0.9792
Epoch: 8 | train_loss: 0.0206 | train_acc: 0.9956 | test_loss: 0.2356 | test_acc: 0.9778
Epoch: 9 | train_loss: 0.0254 | train_acc: 0.9953 | test_loss: 0.3234 | test_acc: 0.9710
Epoch: 10 | train_loss: 0.0298 | train_acc: 0.9948 | test_loss: 0.3633 | test_acc: 0.9760
[INFO] Saving model to: models/vgg11_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1624 | train_acc: 0.9711 | test_loss: 0.0834 | test_acc: 0.9830
Epoch: 2 | train_loss: 0.0368 | train_acc: 0.9906 | test_loss: 0.1218 | test_acc: 0.9755
Epoch: 3 | train_loss: 0.0207 | train_acc: 0.9940 | test_loss: 0.1150 | test_acc: 0.9822
Epoch: 4 | train_loss: 0.0207 | train_acc: 0.9946 | test_loss: 0.1431 | test_acc: 0.9825
Epoch: 5 | train_loss: 0.0190 | train_acc: 0.9954 | test_loss: 0.1688 | test_acc: 0.9840
Epoch: 6 | train_loss: 0.0175 | train_acc: 0.9965 | test_loss: 0.2264 | test_acc: 0.9782
Epoch: 7 | train_loss: 0.0346 | train_acc: 0.9948 | test_loss: 0.2433 | test_acc: 0.9808
Epoch: 8 | train_loss: 0.0375 | train_acc: 0.9953 | test_loss: 0.2757 | test_acc: 0.9830
Epoch: 9 | train_loss: 0.0296 | train_acc: 0.9964 | test_loss: 0.2819 | test_acc: 0.9840
Epoch: 10 | train_loss: 0.0190 | train_acc: 0.9979 | test_loss: 0.4708 | test_acc: 0.9790
[INFO] Saving model to: models/vgg16_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1506 | train_acc: 0.9744 | test_loss: 0.1096 | test_acc: 0.9820
Epoch: 2 | train_loss: 0.0458 | train_acc: 0.9909 | test_loss: 0.2136 | test_acc: 0.9782
Epoch: 3 | train_loss: 0.0282 | train_acc: 0.9932 | test_loss: 0.1313 | test_acc: 0.9860
Epoch: 4 | train_loss: 0.0203 | train_acc: 0.9960 | test_loss: 0.1597 | test_acc: 0.9838
Epoch: 5 | train_loss: 0.0179 | train_acc: 0.9959 | test_loss: 0.1775 | test_acc: 0.9845
Epoch: 6 | train_loss: 0.0248 | train_acc: 0.9955 | test_loss: 0.2827 | test_acc: 0.9832
Epoch: 7 | train_loss: 0.0343 | train_acc: 0.9955 | test_loss: 0.2545 | test_acc: 0.9828
Epoch: 8 | train_loss: 0.0227 | train_acc: 0.9967 | test_loss: 0.4652 | test_acc: 0.9758
Epoch: 9 | train_loss: 0.0266 | train_acc: 0.9971 | test_loss: 0.3563 | test_acc: 0.9832
Epoch: 10 | train_loss: 0.0229 | train_acc: 0.9971 | test_loss: 0.5846 | test_acc: 0.9790
[INFO] Saving model to: models/vgg19_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------